# 932 - 2025

## Problem Statement

For the year $2025$
$$2025 = (20 + 25)^2$$

Given positive integers $a$ and $b$, the concatenation $ab$ we call a $2025$-number if $ab = (a+b)^2$.<br>
Other examples are $3025$ and $81$.<br>
Note $9801$ is not a $2025$-number because the concatenation of $98$ and $1$ is $981$.

Let $T(n)$ be the sum of all $2025$-numbers with $n$ digits or less. You are given $T(4) = 5131$.

Find $T(16)$.

## Solution

The solution to this problem can be obtained with brute force. We loop through all the numbers $n$ from 1 to $10^8$ and check every possible split point in $n^2$. If we find a split such that $ab = (a+b)^2$ and $b$ does not have a leading zero, we add the $n^2$ to the result.


In [11]:
import math
from numba import njit

@njit
def count_digits(n):
    """Returns the number of digits in n."""
    if n == 0:
        return 1
    return math.floor(math.log10(n)) + 1

@njit
def is_2025_number(s):
    """Check if s^2 can be split into two parts whose sum equals s."""
    sq = s * s
    num_digits = count_digits(sq)

    # Try every valid cut position
    divisor = 10
    for _ in range(1, num_digits):  # At least one digit in right part
        a = sq // divisor  # Left part
        b = sq % divisor   # Right part
        if b > 0 and a > 0 and a + b == s and count_digits(a) + count_digits(b) == num_digits:
            return True

        divisor *= 10  # Move to next possible split position

    return False

@njit
def find_2025_numbers(limit):
    """Find and sum all 2025-numbers up to limit."""
    res = 0
    for s in range(1, limit):
        if is_2025_number(s):
            res += s * s
    return res

# Compute result for numbers with at most 16 digits
res = find_2025_numbers(10**8)

res

72673459417881349

Below is a much more efficient solution obtained with the help of ChatGPT o3-mini-high.

In [2]:
import itertools

def prime_factors(n):
    """Return the list of distinct prime factors of n."""
    factors = {}
    d = 2
    temp = n
    while d * d <= temp:
        while temp % d == 0:
            factors[d] = 1
            temp //= d
        d += 1
    if temp > 1:
        factors[temp] = 1
    return list(factors.keys())

def chinese_remainder(remainders, moduli):
    """
    Given lists 'remainders' and 'moduli' (which are pairwise coprime),
    returns the unique solution modulo M = product(moduli) of
         x ≡ r (mod m) for each pair.
    """
    M = 1
    for m in moduli:
        M *= m
    x = 0
    for (r, m) in zip(remainders, moduli):
        M_m = M // m
        # Using Python’s built-in pow with third argument for modular inverse.
        inv = pow(M_m, -1, m)
        x += r * M_m * inv
    return x % M

res = 0

# d = number of digits of b. With d=1,...,8 the total digits of s^2 are at most 16.
for d in range(1, 9):
    m = 10**d - 1   # We require that s(s-1) is divisible by m.
    lower = 10**(d-1)
    upper = 10**d
    if d == 1:
        # Special case: for d=1, the only possibility is s = 9.
        s = 9
        sq = s*s
        # When splitting s^2, the right part must have 1 digit.
        sq_str = str(sq)
        cut = len(sq_str) - 1
        a_str = sq_str[:cut]
        b_str = sq_str[cut:]
        if b_str[0] != '0' and int(a_str) + int(b_str) == s:
            res += sq
    else:
        # For d>=2, we want s in [lower, upper) with s(s-1) ≡ 0 mod m.
        # Note: s(s-1) ≡ 0 mod m means that for every prime factor p of m,
        # we must have s ≡ 0 or 1 (mod p).
        primes = prime_factors(m)
        # The solutions modulo the product of the distinct primes (call it 'step')
        # are given by choosing for each prime factor a residue 0 or 1.
        step = 1
        for p in primes:
            step *= p  # product over distinct primes
        candidate_s = set()
        for combo in itertools.product([0, 1], repeat=len(primes)):
            r_list = list(combo)
            # Solve the system: x ≡ r_i (mod p_i) for each prime factor p_i.
            base = chinese_remainder(r_list, primes)
            # Now, every number of the form s = base + t*step is a solution modulo the product of the distinct primes.
            # We now search for those s in the interval [lower, upper).
            t = 0
            while True:
                s = base + t * step
                if s >= upper:
                    break
                if s >= lower:
                    # Check the full divisibility condition (with the full power of primes in m)
                    if (s*(s-1)) % m == 0:
                        # Now check the “splitting” property:
                        # Write s^2 as a concatenation of two parts, where the right part has exactly d digits.
                        sq = s * s
                        sq_str = str(sq)
                        # The right part must be the last d digits.
                        if len(sq_str) > d:
                            cut = len(sq_str) - d
                            a_str = sq_str[:cut]
                            b_str = sq_str[cut:]
                            if b_str[0] != '0':
                                a_val = int(a_str)
                                b_val = int(b_str)
                                if a_val + b_val == s:
                                    res += s*s
                t += 1

res

72673459417881349